In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import altair as alt

from tools import get_all_files, get_GHG_file_df
from tools.Plot_GHG import get_GHG_plots

In [3]:
# Get the root directory of the data
data_root = '../2023_11_03__17_49_42_hard_mincost_RF30_P1e4_Won_Gon'

# Get all output files
files = get_all_files(data_root)

In [4]:
# Select files based on criteria
files_selc = files.query('year <= 2050')
print(files_selc['catetory'].unique(),'\n')

files_selc

['GHG' 'dvar' 'ammap' 'lmmap' 'lumap_separate' 'lumap' 'water'
 'cross_table' 'quantity'] 



,year,catetory,base_name,base_ext,path
0,2010,GHG,GHG_emissions,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
1,2010,GHG,GHG_emissions_separate_agricultural_landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2,2010,GHG,GHG_emissions_separate_agricultural_management,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
3,2010,GHG,GHG_emissions_separate_no_ag_reduction,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
4,2010,GHG,GHG_emissions_separate_transition_penalty,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
...,...,...,...,...,...
2482,2050,cross_table,switches-irrstat,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2483,2050,cross_table,switches-lmmap,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2484,2050,cross_table,switches-lumap,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...
2485,2050,cross_table,switches-precision-agriculture-amstat,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...


## GHG emissions

In [5]:
# Get the GHG files
GHG_files = get_GHG_file_df(files_selc)
GHG_files = GHG_files.reset_index(drop=True).sort_values(['year','GHG_sum_t'])
GHG_files['GHG_sum_Mt'] = GHG_files['GHG_sum_t'] / 1e6
GHG_files.head(3)

,year,catetory,base_name,base_ext,path,GHG_sum_t,GHG_sum_Mt
1,2010,GHG,Agricultural Management,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.0,0.0
2,2010,GHG,Non-Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.0,0.0
3,2010,GHG,Transition Penalty,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.0,0.0


In [6]:
# Calculate the net emissions
Net_emission = GHG_files.groupby('year')['GHG_sum_Mt'].sum(numeric_only = True).reset_index()
Net_emission = Net_emission.rename(columns={'GHG_sum_Mt':'Net_emission'})
Net_emission.head(3)

,year,Net_emission
0,2010,36.820685
1,2011,-70.476159
2,2012,-70.124035


### 0) Total emissions -- Column chart

In [7]:
# Create a base chart with the necessary transformations and encodings
base_chart = alt.Chart(GHG_files).transform_calculate(
    GHG_sum_Mt = "datum.GHG_sum_t/1000000"
).encode(
    x=alt.X('year:O',axis=alt.Axis(title="Year", labelAngle=-90)),  # Treat year as an ordinal data type
    tooltip=[alt.Tooltip('base_name', title='GHG Category'),
             alt.Tooltip('GHG_sum_Mt:Q', title='Emissions (Mt CO2e)')]
).properties(
    width=600,
    height=400
)



# Create a column chart with the base chart
column_chart = base_chart.mark_bar().encode(
    color=alt.Color('base_name:N',legend=alt.Legend(
                                            title="GHG Category",
                                            orient='none',
                                            legendX=130, legendY=-40,
                                            direction='horizontal',
                                            titleAnchor='middle')),  
    y=alt.Y('GHG_sum_Mt:Q',title='Emissions (Mt CO2e)'),  # Treat GHA_accumulative as a quantitative field
)

# Create a column chart with the base chart
line_chart = alt.Chart(Net_emission).mark_line(color='black').encode(
    tooltip=[alt.Tooltip('Net_emission:Q',title='Net Quantity (Mt CO2e)')],
    x=alt.X('year:O'),
    y=alt.Y('Net_emission:Q',title='Net Quantity (Mt CO2e)'))


# Combine the layers into a final chart
final_chart = alt.layer(
    column_chart,
    line_chart,
).properties(
    width=800,
    height=450
)


final_chart


alt.LayerChart(...)

### 1) Agricultural Land use Emissions

In [19]:
# Initialize the class
ag_lucc_GHG = get_GHG_plots(GHG_files,'Agricultural Landuse','GHG emissions (Mt CO2e)')

  0%|          | 0/41 [00:00<?, ?it/s]

#### 1-1) Emissions by Crops/Lvstk -- Stack column

In [20]:
# get the df and plot
GHG_crop_lvstk_df,GHG_crop_lvstk_plot = ag_lucc_GHG.plot_GHG_crop_lvstk()
GHG_crop_lvstk_plot

alt.LayerChart(...)

#### 1-2) Emissions by dry/irr -- Stack columns

In [ ]:
# get the df and plot
GHG_dry_irr_df,GHG_dry_irr_plot = ag_lucc_GHG.plot_GHG_dry_irr()
GHG_dry_irr_plot

alt.LayerChart(...)

#### 1-3) Emissions by lu-lm cmobined

In [ ]:
# get the df and plot
GHG_lu_lm_df,GHG_lu_lm_plot = ag_lucc_GHG.plot_GHG_lu_lm(2050)
GHG_lu_lm_plot

alt.Chart(...)

#### 1-4) Emissions by lu-sources combined

In [ ]:
# get the df and plot
GHG_lu_source_df,GHG_lu_source_plot = ag_lucc_GHG.plot_GHG_lu_source(2050)
GHG_lu_source_plot

alt.Chart(...)

### 2) Non-agricultural Land use sequestrations

In [ ]:
# Initialize the class
non_ag_lucc_GHG = get_GHG_plots(GHG_files,'Non-Agricultural Landuse','GHG sequestration (Mt CO2e)')

  0%|          | 0/41 [00:00<?, ?it/s]

In [ ]:
GHG_type = 'Non-Agricultural Landuse'

In [ ]:
GHG_files = GHG_files.query(f'base_name == "{GHG_type}"').reset_index(drop=True)
GHG_files.head(3)

,year,catetory,base_name,base_ext,path,GHG_sum_t,GHG_sum_Mt
0,2010,GHG,Non-Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,0.0,0.000000
1,2011,GHG,Non-Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,-109002744.0,-109.002744
2,2012,GHG,Non-Agricultural Landuse,.csv,../2023_11_03__17_49_42_hard_mincost_RF30_P1e4...,-104003110.0,-104.003110


In [ ]:
# Read GHG emissions of ag lucc
import pandas as pd
from tqdm import tqdm

from PARAMETERS import LU_CROPS, LU_lvstk


CSVs = []
for _,row in tqdm(GHG_files.iterrows(),total=GHG_files.shape[0]):
    csv = pd.read_csv(row['path'],index_col=0,header=[0,1,2]).drop('SUM',axis=1)

    csv_crop = csv[[True if i in LU_CROPS else False for i in  csv.index]]
    csv_crop.index = pd.MultiIndex.from_product(([row['year']], csv_crop.index, ['Crop']))

    csv_lvstk = csv[[True if i in LU_lvstk else False for i in  csv.index]]
    csv_lvstk.index = pd.MultiIndex.from_tuples(tuple(csv_lvstk.index.str.split(' - ')))
    csv_lvstk = csv_lvstk.groupby(level=0).sum(numeric_only=True)
    csv_lvstk.index = pd.MultiIndex.from_product(([row['year']], csv_lvstk.index, ['Livestock']))

    csv = pd.concat([csv_crop,csv_lvstk],axis=0)
    CSVs.append(csv)

# Convert the GHG to long format, so that each variable is in one column
GHG_df = pd.concat(CSVs,axis=0)
GHG_df = GHG_df.droplevel(0,axis=1)

GHG_df_long = GHG_df.copy()
GHG_df_long.columns = ["+".join(i) for i in GHG_df_long.columns.tolist()]
GHG_df_long = GHG_df_long.reset_index()
GHG_df_long.columns = ['Year','Land use','Land use category'] + GHG_df_long.columns.tolist()[3:]

GHG_df_long = GHG_df_long.melt(id_vars=['Year','Land use','Land use category'],
                                    value_vars=GHG_df_long.columns.tolist()[3:],
                                    value_name='val_t')


GHG_df_long['Quantity (Mt CO2e)'] = GHG_df_long['val_t'] / 1e6
GHG_df_long[['Irrigation','Sources']] = GHG_df_long['variable'].str.split('+',expand=True)
GHG_df_long.drop(['val_t','variable'],axis=1,inplace=True)
GHG_df_long = GHG_df_long.reindex(columns=['Year','Land use category','Land use',
                                                'Irrigation','Sources', 'Quantity (Mt CO2e)'])

100%|██████████| 41/41 [00:00<00:00, 137.99it/s]


In [ ]:
csv = pd.read_csv(row['path'],index_col=0,header=[0,1,2]).drop('SUM',axis=1)

In [ ]:
csv_crop = csv[[True if i in LU_CROPS else False for i in  csv.index]]
csv_crop.index = pd.MultiIndex.from_product(([row['year']], csv_crop.index, ['Crop']))

csv_lvstk = csv[[True if i in LU_lvstk else False for i in  csv.index]]
csv_lvstk.index = pd.MultiIndex.from_tuples(tuple(csv_lvstk.index.str.split(' - ')))
csv_lvstk = csv_lvstk.groupby(level=0).sum(numeric_only=True)
csv_lvstk.index = pd.MultiIndex.from_product(([row['year']], csv_lvstk.index, ['Livestock']))

csv = pd.concat([csv_crop,csv_lvstk],axis=0)

In [ ]:
csv

Non_Agricultural Landuse  \
                                                     dry   
                            TCO2E_Environmental Planting   
Beef - modified land                        0.000000e+00   
Beef - natural land                         0.000000e+00   
Cotton                                      0.000000e+00   
Dairy - modified land                       0.000000e+00   
Dairy - natural land                        0.000000e+00   
Environmental Plantings                    -1.001595e+08   
Grapes                                      0.000000e+00   
Hay                                         0.000000e+00   
Nuts                                       -3.386938e+05   
Other non-cereal crops                      0.000000e+00   
Rice                                        0.000000e+00   
Sheep - modified land                       0.000000e+00   
Sheep - natural land                        0.000000e+00   
Sugar                                      -7.791604e+04   
Summer cereals                              0.000000e+00   
Unallocated - modified land                -1.948227e+05   
Unallocated - natural land                  0.000000e+00   
Vegetables                                  0.000000e+00   
Winter cereals                              0.000000e+00   
Winter legumes                              0.000000e+00   
Winter oilseeds                             0.000000e+00   
SUM                                        -1.007710e+08   

                                                          
                                                     irr  
                            TCO2E_Environmental Planting  
Beef - modified land                                 0.0  
Beef - natural land                                  0.0  
Cotton                                               0.0  
Dairy - modified land                                0.0  
Dairy - natural land                                 0.0  
Environmental Plantings                       -4900880.5  
Grapes                                               0.0  
Hay                                                  0.0  
Nuts                                                 0.0  
Other non-cereal crops                               0.0  
Rice                                                 0.0  
Sheep - modified land                                0.0  
Sheep - natural land                                 0.0  
Sugar                                                0.0  
Summer cereals                                       0.0  
Unallocated - modified land                          0.0  
Unallocated - natural land                           0.0  
Vegetables                                           0.0  
Winter cereals                                       0.0  
Winter legumes                                       0.0  
Winter oilseeds                                      0.0  
SUM                                           -4900880.5

In [ ]:
csv

Non_Agricultural Landuse  \
                                                               dry   
                                      TCO2E_Environmental Planting   
2050 Cotton                 Crop                              0.00   
     Grapes                 Crop                              0.00   
     Hay                    Crop                              0.00   
     Nuts                   Crop                        -338693.84   
     Other non-cereal crops Crop                              0.00   
     Rice                   Crop                              0.00   
     Sugar                  Crop                         -77916.04   
     Summer cereals         Crop                              0.00   
     Vegetables             Crop                              0.00   
     Winter cereals         Crop                              0.00   
     Winter legumes         Crop                              0.00   
     Winter oilseeds        Crop                              0.00   
     Beef                   Livestock                         0.00   
     Dairy                  Livestock                         0.00   
     Sheep                  Livestock                         0.00   

                                                                    
                                                               irr  
                                      TCO2E_Environmental Planting  
2050 Cotton                 Crop                               0.0  
     Grapes                 Crop                               0.0  
     Hay                    Crop                               0.0  
     Nuts                   Crop                               0.0  
     Other non-cereal crops Crop                               0.0  
     Rice                   Crop                               0.0  
     Sugar                  Crop                               0.0  
     Summer cereals         Crop                               0.0  
     Vegetables             Crop                               0.0  
     Winter cereals         Crop                               0.0  
     Winter legumes         Crop                               0.0  
     Winter oilseeds        Crop                               0.0  
     Beef                   Livestock                          0.0  
     Dairy                  Livestock                          0.0  
     Sheep                  Livestock                          0.0

#### 2-1) Sequestrations by Crop/Lvstk -- stack column

In [ ]:
# get the df and plot
GHG_non_ag_crop_lvstk_df,GHG_non_ag_crop_lvstk_plot = non_ag_lucc_GHG.plot_GHG_crop_lvstk()
GHG_non_ag_crop_lvstk_plot

alt.LayerChart(...)

#### 2-2) Sequestrations by dry/irr -- stack columns

In [ ]:
# get the df and plot
GHG_non_ag_dry_irr_df,GHG_non_ag_dry_irr_plot = non_ag_lucc_GHG.plot_GHG_dry_irr()
GHG_non_ag_dry_irr_plot

alt.LayerChart(...)